In [1]:
import os
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
from datetime import datetime,timedelta
from openpyxl.styles import NamedStyle,PatternFill,numbers
from openpyxl.utils import get_column_letter

In [2]:
# Change directory to new_dir
current_dir = os.getcwd()
new_dir = r'C:\\Users\\user\\Desktop'
os.chdir(new_dir)
current_dir = os.getcwd()
print("Current Directory after directory change:", current_dir)
# List files inside the directory
Files_inside_dir = os.listdir()
print("Files inside Directory:", Files_inside_dir)
# Construct the filenames
file_lawson = "tss3333ddsinq.xlsx"
print("Constructed Lawson filename:", file_lawson)
file_mapping = "Mapping Tabs for Lawson v9-12-24.xlsx"
print("Constructed Lawson mapping filename:", file_mapping)
# Check if the constructed filenames are in the directory
if file_lawson in Files_inside_dir and file_mapping in Files_inside_dir:
    print("All required files found in directory.")
else:
    print("One or more required files not found in directory.")
print("Once the files are read correctly, the process of copying and writing into Lawson file starts from here onwards.")
# Opening Source file
source_wb = load_workbook(file_lawson, data_only=True)
source_ws = source_wb.active
# Load mappings for CO, AU, Region, Acct Desc, and HRS/FTE
mapping_df_co = pd.read_excel(file_mapping, sheet_name='CoMap', usecols="A:B", skiprows=2)
mapping_dict_co = pd.Series(mapping_df_co.iloc[:, 1].values, index=mapping_df_co.iloc[:, 0]).to_dict()
mapping_df_co_au = pd.read_excel(file_mapping, sheet_name='Co_AuMap', usecols="A:E", skiprows=1)
mapping_dict_co_au = pd.Series(mapping_df_co_au.iloc[:, 4].values, index=mapping_df_co_au.iloc[:, 0]).to_dict()
mapping_df_co_region = pd.read_excel(file_mapping, sheet_name='CoMap', usecols="A:D", skiprows=2)
mapping_dict_co_region = pd.Series(mapping_df_co_region.iloc[:, 3].values, index=mapping_df_co_region.iloc[:, 0]).to_dict()
mapping_df_acct = pd.read_excel(file_mapping, sheet_name='AcctMap', usecols="A:B", skiprows=1)
mapping_dict_acct_desc = pd.Series(mapping_df_acct.iloc[:, 1].values, index=mapping_df_acct.iloc[:, 0]).to_dict()
mapping_df_ftehrs = pd.read_excel(file_mapping, sheet_name='FteHrsMap', usecols="B:F", skiprows=1)
mapping_dict_ftehrs = pd.Series(mapping_df_ftehrs.iloc[:, 4].values, index=mapping_df_ftehrs.iloc[:, 0]).to_dict()
# Rename columns and add new columns as specified
for cell in source_ws[1]:
    if cell.value == "CORP":
        cell.value = "CO"
    elif cell.value == "CC":
        cell.value = "AU"
source_ws.insert_cols(2, amount=3)
source_ws.insert_cols(6, amount=3)
source_ws.insert_cols(10, amount=2)
source_ws.cell(row=1, column=2).value = "CO DESC"
source_ws.cell(row=1, column=3).value = "CMI CO"
source_ws.cell(row=1, column=4).value = "CMI CO DESC"
source_ws.cell(row=1, column=6).value = "AU DESC"
source_ws.cell(row=1, column=7).value = "REGION"
source_ws.cell(row=1, column=8).value = "CONCAT"
source_ws.cell(row=1, column=10).value = "ACCT DESC"
source_ws.cell(row=1, column=11).value = "HRS/FTE"
source_ws.cell(row=1, column=24).value = "YTD"
accounting_style = NamedStyle(name="accounting_style", number_format="#,##0.00_);(#,##0.00)")
source_wb.add_named_style(accounting_style)
for row in range(2, source_ws.max_row + 1):
    co_value = source_ws.cell(row=row, column=1).value
    au_value = source_ws.cell(row=row, column=5).value 
    concat_value = f"{co_value}{au_value}"
    source_ws.cell(row=row, column=8).value = concat_value  
    co_desc = mapping_dict_co.get(co_value, "#N/A")
    source_ws.cell(row=row, column=2).value = co_desc 
    concat_value_int = int(concat_value)
    au_desc = mapping_dict_co_au.get(concat_value_int, "#N/A")
    source_ws.cell(row=row, column=6).value = au_desc 
    region_value = mapping_dict_co_region.get(co_value, "#N/A")
    source_ws.cell(row=row, column=7).value = region_value
    acct_value = source_ws.cell(row=row, column=9).value
    acct_desc_value = mapping_dict_acct_desc.get(acct_value, "#N/A")
    source_ws.cell(row=row, column=10).value = acct_desc_value  
    hrs_fte_value = mapping_dict_ftehrs.get(acct_value, "#N/A")
    source_ws.cell(row=row, column=11).value = hrs_fte_value  
    source_ws.cell(row=row, column=3).value = source_ws.cell(row=row, column=1).value
    source_ws.cell(row=row, column=4).value = source_ws.cell(row=row, column=2).value
    sum_value = 0
    for col in range(12, 24):
        cell_value = source_ws.cell(row=row, column=col).value
        if cell_value is not None:
            sum_value += cell_value
    sum_value = round(sum_value, 2)
    source_ws.cell(row=row, column=24).value = sum_value
# Apply the custom style to columns L to X (columns are 12 to 24)
for col in range(12, 25):  # Columns L to X are 12 to 24 in Excel (1-based index)
    for row in range(2, source_ws.max_row + 1):
        cell = source_ws.cell(row=row, column=col)
        cell.style = accounting_style
# Reorder columns: Move REGION (column 7) to column 1, CONCAT (column 8) to column 2, and CO (column 1) to column 3
def move_column(ws, col_from, col_to):
    """ Move a column from col_from index to col_to index in the worksheet ws """
    ws.insert_cols(col_to)
    for row in range(1, ws.max_row + 1):
        ws.cell(row=row, column=col_to).value = ws.cell(row=row, column=col_from + 1).value
    ws.delete_cols(col_from + 1)
# Move REGION to column 1 (originally at column 7)
move_column(source_ws, 7, 1)
# Move CONCAT to column 2 (originally at column 8)
move_column(source_ws, 8, 2)
# Move CO to column 3 (originally at column 1)
move_column(source_ws, 1, 3)
# Resize the columns to fit the contents
max_width = 30  # Define a maximum width for columns
min_width = 10  # Define a minimum width for columns
for col in source_ws.columns:
    max_length = 0
    column = col[0].column_letter  # Get the column name
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = min(max_length + 2, max_width)
    adjusted_width = max(adjusted_width, min_width)
    source_ws.column_dimensions[column].width = adjusted_width
# Define the fill color
fill_color = PatternFill(start_color="FFF4CCCC", end_color="FFF4CCCC", fill_type="solid")
# Apply color to columns A to K excluding C, G, and I
for col in range(1, 12):
    if col not in [3, 7, 9]:  # Exclude columns C (3), G (7), I (9)
        column_letter = get_column_letter(col)
        for row in range(2, source_ws.max_row + 1):  # Start from row 2 to exclude header
            source_ws[f"{column_letter}{row}"].fill = fill_color
'''
# Filter and copy operation
filtered_rows = []
for row in range(2, source_ws.max_row + 1):
    cmi_co_value = source_ws.cell(row=row, column=3).value
    au_value = source_ws.cell(row=row, column=7).value
    if cmi_co_value in [5201, 5202] and au_value in [91000, 91001]:
        filtered_rows.append(row)
# Copy CO value: 5202 and AU DESC: STATISTICS to CO: 5200 and AU DESC: STATISTICS NICU
for row in filtered_rows:
    co_value = source_ws.cell(row=row, column=1).value
    if co_value == 5200:
        source_ws.cell(row=row, column=1).value = 5202
        source_ws.cell(row=row, column=6).value = "MCLANE CHILDREN'S HOSPITAL"
 # Apply yellow fill to indicate the change
        for col in range(1, 7):  # Adjust the range as per your columns
            source_ws.cell(row=row, column=col).fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
'''
# Save the updated workbook with the specified filename
# Get the current month and year
current_date = datetime.now()
# Calculate the previous month
prev_month_date = current_date.replace(day=1)- timedelta(days=1) 
prev_month_short = prev_month_date.strftime("%b") 
print("Name of the month: ",prev_month_short)
current_year = current_date.strftime("%Y")
current_year_short = current_date.strftime("%y")
timestamp = datetime.now().strftime("%Y%m%d_%I_%M_%S_%p")
file_lawson_save = f"Lawson All Units {prev_month_short} FY{current_year_short} Prelims_{timestamp}.xlsx"
source_wb.save(file_lawson_save)
print("Lawson file updated successfully.")

Current Directory after directory change: C:\Users\user\Desktop
Files inside Directory: ['2024 MRVU August.xlsx', '950680_APR_05072024.xlsx', 'Aug 2024 Consolidated Stats.xlsx', 'August pat 2024 numbers.xlsx', 'BUMC_FY25 Monthly Bed Report.xlsx', 'CMI & CWAD Calculations April 2024.xlsx', 'CMICWADMOYTD.xlsx', 'desktop.ini', 'HealthCare Basics.txt', 'Hospitality Statistics - AUG FY24 - NOLINKS.xlsx', 'Labor hours - August 2024.xlsx', 'Laundry_MASTER_FY2025.xlsx', 'Manual JE Codes_Updated', 'Mapping Tabs for Lawson v5-16-24.xlsx', 'Mapping Tabs for Lawson v9-12-24.xlsx', 'MIDAS CMI By Payor APR.xlsx', 'Month End Numbers August 24.xlsx', 'OUTREACH RVU FY25.xlsx', 'Scorecard FY24(5140.10590).xlsx', 'TouchStone_RVUs-Extract.xlsx', 'Transport Volumes Monthly FY2025.xlsx', 'tss3333ddsinq.xlsx']
Constructed Lawson filename: tss3333ddsinq.xlsx
Constructed Lawson mapping filename: Mapping Tabs for Lawson v9-12-24.xlsx
All required files found in directory.
Once the files are read correctly, the 